# MixMatch PyTorch Implementation

## 필요한 패키지 다운로드

In [1]:
! pip install colorama
! git clone https://github.com/hansam95/LG-Elec-Day18.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


## 필요한 패키지 중 이미 다운 받아진 패키지 부르기

In [2]:
import os, math, sys, argparse
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from tqdm import tqdm
from colorama import Fore
from torch.utils.data import DataLoader
from torchvision import transforms as transforms

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def Normalize(x, m=(0.4914, 0.4822, 0.4465), std=(0.2471, 0.2345, 0.2616)):
    # m: mean, std: standard deviation
    # (mean, std) 사전에 구해진 값이기에 따로 구할 필요 없음
    # MixMatch를 개인적으로 사용하시는 분은 새롭게 구해야 함
    # ex: 변수가 10개인 tablur data에 적용한다면, 변수 별 mean, std를 구해주어야함
    # from torchvision.transforms 내 Normalize 함수와 동일한 함수
    x, m, std = [np.array(a, np.float32) for a in (x, m, std)]

    x -= m * 255
    x *= 1.0/(255*std)
    return x


def Transpose(x, source='NHWC', target='NCHW'):
    # N, H, W, C = Batch_size, Height, Width, # of channels
    # 일반적인 이미지의 경우 3
    # torch.nn.Conv2d는 (N, C, H, W) 의 형태를 가진 데이터를 입력 받기 때문에 형태 변경
    # from torchvision.transforms 내 ToTensor 와 동일한 함수
    return x.transpose([source.index(d) for d in target])


def pad(x, border=4):
    # 특정 이미지에 동서남북 방향으로 4만큼 픽셀을 추가해 붙여주는 작업
    # 이를 padding이라 부름
    return np.pad(x, [(0, 0), (border, border), (border, border)], mode='reflect')

In [4]:
class Labeled_CIFAR10(torchvision.datasets.CIFAR10):
    def __init__(self, root, indices=None,
                train=True, transform=None,
                target_transform=None, download=False):
        super(Labeled_CIFAR10, self).__init__(root,
                                        train=train,
                                        transform=transform,
                                        target_transform=target_transform,
                                        download=download)

        # label 데이터로 지정된 index
        if indices is not None:
            self.data = self.data[indices]
            self.targets = np.array(self.targets)[indices]
        
        self.data = Transpose(Normalize(self.data))
    
    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        if self.transform is not None:
            img = self.transform(img)
        
        if self.target_transform is not None:
            target = self.target_transform(target)
        
        return img, target

In [5]:
class ToTensor(object):
    def __call__(self, x):
        x = torch.from_numpy(x)
        return x

## WideResNet

In [6]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0, activate_before_residual=False):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
        self.activate_before_residual = activate_before_residual
    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual == True:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)
    

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0, activate_before_residual=False):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate, activate_before_residual)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate, activate_before_residual):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate, activate_before_residual))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)
    

class WideResNet(nn.Module):
    def __init__(self, num_classes, depth=28, widen_factor=2, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor] # 채널수
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate, activate_before_residual=True)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.zero_()

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)

In [7]:
def get_tqdm_config(total, leave=True, color='white'):
    fore_colors = {
        'red': Fore.LIGHTRED_EX,
        'green': Fore.LIGHTGREEN_EX,
        'yellow': Fore.LIGHTYELLOW_EX,
        'blue': Fore.LIGHTBLUE_EX,
        'magenta': Fore.LIGHTMAGENTA_EX,
        'cyan': Fore.LIGHTCYAN_EX,
        'white': Fore.LIGHTWHITE_EX,
    }
    return {
        'file': sys.stdout,
        'total': total,
        'desc': " ",
        'dynamic_ncols': True,
        'bar_format':
            "{l_bar}%s{bar}%s| [{elapsed}<{remaining}, {rate_fmt}{postfix}]" % (fore_colors[color], Fore.RESET),
        'leave': leave
    }

## Evaluation metric
#### top1 accuracy, top5 accuracy
#### top1 accuracy: (확률 값이 가장 높은 범주와 실제 범주가 일치하는 관측치 수)/ 전체 관측치
#### top5 accuracy: (확률 값 상위 5개 중 실제 범주가 존재하는 관측치 수)/ 전체 관측치

In [8]:
def accuracy(output, target, topk=(1, )):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        if k == 1:
            correct_k = correct[:k].view(-1).float().sum(0)
        if k > 1:
            correct_k = correct[:k].float().sum(0).sum(0)
        acc = correct_k.mul_(100.0 / batch_size)
        acc = acc.detach().cpu().numpy()
        res.append(acc)
    return res

In [ ]:
class MixMatchTester():
    def __init__(self, args):
        self.args = args

        # root_dir = './MixMatch'
        root_dir = '/content/LG-Elec-Day18/MixMatch'

        # Data
        print("==> Preparing CIFAR10 dataset")
        transform_val = transforms.Compose([
            ToTensor()
        ]) # validation, test dataset에 대한 data augmentation 정의
           # 합성곱 신경망에 입력 될 수 있도록만 지정(Augmentation 사용하지 않는 것과 동일)

        test_set = Labeled_CIFAR10(os.path.join(root_dir, 'data'), train=False, transform=transform_val, download=True) # test dataset
        
        # DataLoader 정의
        self.test_loader = DataLoader(
            dataset=test_set, shuffle=False, num_workers=0, drop_last=False
        )

        # Build WideResNet
        print("==> Preparing WideResNet")
        self.model = self.create_model(ema=False)
        self.ema_model = self.create_model(ema=True)

        ckpt_path = os.path.join(root_dir, 'pretrain', 'ema_model.pth')
        ckpt = torch.load(ckpt_path)
        self.ema_model.load_state_dict(ckpt)

    def create_model(self, ema=False):
        # Build WideResNet & EMA model
        model = WideResNet(num_classes=10)
        model = model.to(self.args.cuda)

        if ema:
            for param in model.parameters():
                param.detach_()
            
        return model

    @torch.no_grad()
    def validate(self):
        self.ema_model.eval()     
        data_loader = self.test_loader
        c = 'red'

        top1s, top5s = [], []

        with tqdm(**get_tqdm_config(total=len(data_loader),
                leave=True, color=c)) as pbar:
            for batch_idx, (inputs, targets) in enumerate(data_loader):
                inputs, targets = inputs.to(self.args.cuda), targets.to(self.args.cuda)

                outputs = self.ema_model(inputs)
                # labeled dataset에 대해서만 손실함수 계산
                # torch.nn.CrossEntropyLoss()를 사용해서 손실함수 계산

                prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
                top1s.append(prec1)
                top5s.append(prec5)

                pbar.set_description(
                    '[Top1 Acc: %.3f|Top5 Acc: %.3f]'%(np.mean(top1s), np.mean(top5s))
                )
                pbar.update(1)

            pbar.set_description(
                '[Top1 Acc: %.3f|Top5 Acc: %.3f]'%(np.mean(top1s), np.mean(top5s))
            )

        return np.mean(top1s), np.mean(top5s)

In [12]:
class FixMatchTester():
    def __init__(self, args):
        self.args = args

        root_dir = './FixMatch'
        root_dir = '/content/LG-Elec-Day18/FixMatch'

        # Data
        print("==> Preparing CIFAR10 dataset")
        transform_val = transforms.Compose([
            ToTensor()
        ]) # validation, test dataset에 대한 data augmentation 정의
           # 합성곱 신경망에 입력 될 수 있도록만 지정(Augmentation 사용하지 않는 것과 동일)

        test_set = Labeled_CIFAR10(os.path.join(root_dir, 'data'), train=False, transform=transform_val, download=True) # test dataset
        
        # DataLoader 정의
        self.test_loader = DataLoader(
            dataset=test_set, shuffle=False, num_workers=0, drop_last=False
        )

        # Build WideResNet
        print("==> Preparing WideResNet")
        self.ema_model = WideResNet(self.args.n_classes).to(self.args.cuda)

        ckpt_path = os.path.join(root_dir, 'pretrain', 'ema_model.pth')
        ckpt = torch.load(ckpt_path)
        self.ema_model.load_state_dict(ckpt)
    
    # MixMatch validate 함수와 동일
    @torch.no_grad()
    def validate(self):
        self.ema_model.eval()     
        data_loader = self.test_loader
        c = 'red'

        top1s, top5s = [], []

        with tqdm(**get_tqdm_config(total=len(data_loader),
                leave=True, color=c)) as pbar:
            for batch_idx, (inputs, targets) in enumerate(data_loader):
                inputs, targets = inputs.to(self.args.cuda), targets.to(self.args.cuda)

                outputs = self.ema_model(inputs)
                # labeled dataset에 대해서만 손실함수 계산
                # torch.nn.CrossEntropyLoss()를 사용해서 손실함수 계산

                prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
                top1s.append(prec1)
                top5s.append(prec5)

                pbar.set_description(
                    '[Top1 Acc: %.3f|Top5 Acc: %.3f]'%(np.mean(top1s), np.mean(top5s))
                )
                pbar.update(1)

            pbar.set_description(
                '[Top1 Acc: %.3f|Top5 Acc: %.3f]'%(np.mean(top1s), np.mean(top5s))
            )

        return np.mean(top1s), np.mean(top5s)

In [13]:
def MixMatch_parser():
    parser = argparse.ArgumentParser(description="MixMatch PyTorch Implementation for LG Electornics education")
    
    # method arguments
    parser.add_argument('--n-labeled', type=int, default=1000)
    parser.add_argument('--num-iter', type=int, default=1024,
                        help="The number of iteration per epoch")
    parser.add_argument('--alpha', type=float, default=0.75)
    parser.add_argument('--lambda-u', type=float, default=75)
    parser.add_argument('--T', default=0.5, type=float)
    parser.add_argument('--ema-decay', type=float, default=0.999)

    parser.add_argument('--epochs', type=int, default=3)
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument('--lr', type=float, default=0.002)

    return parser

In [14]:
def FixMatch_parser():
    parser = argparse.ArgumentParser(description="FixMatch PyTorch Implementation for LG Electornics education")
    
    # method arguments
    parser.add_argument('--n-labeled', type=int, default=3000)
    parser.add_argument('--n-classes', type=int, default=10)
    parser.add_argument("--expand-labels", action="store_true",
                        help="expand labels to fit eval steps")

    # training hyperparameters
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument('--total-steps', default=2**20, type=int)
    parser.add_argument('--eval-step', type=int, default=1024)
    parser.add_argument('--lr', type=float, default=0.03)
    parser.add_argument('--weight-decay', type=float, default=5e-4)
    parser.add_argument('--nesterov', action='store_true', default=True)
    parser.add_argument('--warmup', type=float, default=0.0)

    parser.add_argument('--use-ema', action='store_true', default=True)
    parser.add_argument('--ema-decay', type=float, default=0.999)

    parser.add_argument('--mu', type=int, default=7)
    parser.add_argument('--T', type=float, default=1.0)

    parser.add_argument('--threshold', type=float, default=0.95)
    parser.add_argument('--lambda-u', type=float, default=1.0)
    return parser

In [20]:
parser = MixMatch_parser()
args = parser.parse_args([])
args.cuda = torch.device("cuda:0")
tester = MixMatchTester(args)
tester.validate()

==> Preparing CIFAR10 dataset
Files already downloaded and verified
==> Preparing WideResNet
[Top1 Acc: 91.450|Top5 Acc: 99.350]: 100%|██████████| [00:22<00:00, 442.34it/s]  


(91.45, 99.35)

In [15]:
parser = FixMatch_parser()
args = parser.parse_args([])
args.cuda = torch.device("cuda:0")
tester = FixMatchTester(args)
tester.validate()

==> Preparing CIFAR10 dataset
Files already downloaded and verified
==> Preparing WideResNet
[Top1 Acc: 39.490|Top5 Acc: 87.430]: 100%|██████████| [00:22<00:00, 443.52it/s] 


(39.49, 87.43)